In [1]:
# Template Notebook for Engagement Classifcation Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [30]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random

In [3]:
# Load Data
# Warning: this will probably take some time
# Adjust file path based on your computer

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y9,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,8.0,1.0,2018-06-07,0.000000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,8.0,1.0,2018-06-07,0.033333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
2,8.0,1.0,2018-06-07,0.066667,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
3,8.0,1.0,2018-06-07,0.100000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
4,8.0,1.0,2018-06-07,0.133333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [5]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity', 'of_face_id'])

In [6]:
# Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' in i or 'of_' in i:
        only_ofop.append(i)
    else:
        performance.append(i)
        
only_ofop.remove('of_timestamp')

In [7]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Step 1: Set Up Feature Sets

1. All OF/OP + Performance
2. Performance ONLY
3. All OF/OP ONLY
4. Performance + Handpicked OF/OP (from Caitlyn's intuition)
5. Performance + Selected OF/OP (from Feature Analysis) 

Note: keep timestamp and session_num until right before running the model in all feature sets

In [8]:
# Feature Set 1: All OF/OP + Performance

FS1 = data

In [9]:
# Feature Set 2: Performance ONLY

print(performance)

FS2 = data[performance]

['session_num', 'timestamp', 'engagement', 'aptitude', 'games_total', 'games_session', 'mistakes_total', 'mistakes_session', 'mistakes_game', 'ts_robot_talked', 'ts_game_start', 'ts_attempt', 'skill_NC', 'skill_OS', 'skill_EM', 'diff_1', 'diff_2', 'diff_3', 'diff_4', 'diff_5', 'no_game']


In [10]:
# Feature Set 3: OF/OP ONLY  

only_ofop.append('timestamp')
only_ofop.append('session_num')
only_ofop.append('engagement')

FS3 = data[only_ofop]

In [11]:
# Feature Set 4: Performance + Handpicked OF/OP

hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance:
    hand_picked.append(i)

FS4 = data[hand_picked]

In [12]:
# Feature Set 5: Performance + Selected OF/OP

selected_ofop = ['of_AU04_c',   'of_AU06_c',    'of_AU07_c',    'of_AU14_c',    'of_AU15_c',    'of_AU25_c',    'of_confidence',    'of_frame',    'of_success',    'of_timestamp', 'op_p1_face_c25',    'op_p1_face_c43',    'op_p1_face_c44',    'op_p1_face_c45',    'op_p1_face_c46',    'op_p1_face_y7',    'op_p1_face_y8',    'op_p1_pose_c0',    'op_p1_pose_c15',    'op_p1_pose_c16']

for i in performance:
    selected_ofop.append(i)

FS5 = data[selected_ofop]

### Step 2: Create Windows for RNN (for chosen Feature Set)

In [22]:
# CHOOSE FEATURE SET, Window Size

FS = FS1

# window_size = # of frames to feed into an RNN at a time
window_size = 50 

session_threshold = 8 # train on sessions <= 8, test on sessions > 8

In [14]:
# Sort Data (Temporal Model)

FS = FS.sort_values(['session_num', 'timestamp'])

In [29]:
# Create Windows, mapped by session
# throw away end/remainder of session < 50 frames

windows = []
X_train2 = []
y_train2 = []
X_test2 = []
y_test2 = []

curr_session = -1
window_start = -1
curr_window = -1

for i,r in FS.iterrows():
    this_session = r['session_num']
    
    # we've encountered a new session
    if curr_session != this_session:
        # ignore remainder window
        curr_session = this_session
        window_start = i
        curr_window = 1
        continue
    
    # we've found a complete window
    if curr_window == window_size - 1:
        to_add = FS[window_start:i+1]
        to_add = to_add.drop(columns=['session_num', 'timestamp'])
        
        windows.append(to_add)
        if (curr_session <= session_threshold):
            y_train2.append(to_add['engagement'])
            X_train2.append(to_add.drop(columns=['engagement']))
        else:
            y_test2.append(to_add['engagement'])
            X_test2.append(to_add.drop(columns=['engagement']))
            
        window_start = i+1
        curr_window = 0
        continue 
        
    curr_window += 1

In [31]:
# Shuffle Windows

random.shuffle(windows)

y1 = []
X1 = []

for i in windows:
    y1.append(i['engagement'])
    X1.append(i.drop(columns=['engagement']))

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=42)

### Step 3: Implement Your Model!

In [32]:
# For chosen feature set, run on both Train-Test Splits
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.optimizers import RMSprop
from keras import metrics
from keras.models import Model

Using TensorFlow backend.


In [40]:
len(X_train1)

3635

In [38]:
model = Sequential()

# At this point, each individual training sample is now a sequence of word embedding vectors

num_features = X_train1[0].shape[1]

model.add(LSTM(64, input_length=window_size, input_dim=num_features, return_sequences = False)) #return_sequences: if multiple LSTM layers
model.add(Dropout(.2))

# Dense 2 (final vote)
model.add(Dense(50, activation = 'sigmoid'))

print(model.summary())

# Define Loss and Optimizer
LOSS = 'binary_crossentropy' #classifying between 0 and 1
OPTIMIZER = 'rmsprop' #RMSprop typically works better for RNNs per Keras

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

/anaconda3/envs/caispp/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/anaconda3/envs/caispp/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=False, input_shape=(50, 1554))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                414464    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                3250      
Total params: 417,714
Trainable params: 417,714
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
EPOCHS = 10
BATCH_SIZE = 64

model.fit(X_train1, y_train1, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =.2)


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 3635 arrays: [array([[0.3       , 0.09803922, 0.3125    , ..., 0.        , 0.        ,
        0.        ],
       [0.3       , 0.09803922, 0.3125    , ..., 0.        , 0.        ,
        0.        ],
       [0.3...